In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='4,5,6,7'

In [ ]:
%run -p ../train_cnf_disentangle_rl.py --data cifar10 --dims 64,64,64 --strides 1,1,1,1 --num_blocks 2 --layer_type concat --multiscale True --rademacher True --batch_size 900 --test_batch_size 500 --save ../experiments_published/cnf_conditional_disentangle_cifar10_bs900_sratio_0_5_drop_0_5_rl_stdscale_15_annealing_run3 --seed 3 --lr 0.001 --conditional True --controlled_tol False --train_mode semisup --log_freq 10 --weight_y 0.5 --condition_ratio 0.5 --dropout_rate 0.5 --scale_fac 1.0 --scale_std 15.0 --annealing_std True
#

/tancode/repos/tan-ffjord/train_cnf_disentangle_rl.py
import argparse
import os
import time
import numpy as np

import torch
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as tforms
from torchvision.utils import save_image

import lib.layers as layers
import lib.utils as utils
import lib.multiscale_parallel as multiscale_parallel
import lib.modules as modules
import lib.thops as thops

from train_misc import standard_normal_logprob
from train_misc import set_cnf_options, count_nfe, count_parameters, count_total_time, count_nfe_gate
from train_misc import add_spectral_norm, spectral_norm_power_iteration
from train_misc import create_regularization_fns, get_regularization, append_regularization_to_log

from tensorboardX import SummaryWriter

# go fast boi!!
torch.backends.cudnn.benchmark = True
SOLVERS = ["dopri5", "bdf", "rk4", "midpoint", 'adams', 'explicit_adams']
GATES = ["cnn1", "cnn2", "rnn"]

parser = argparse.ArgumentParser("Continuo

Files already downloaded and verified
Files already downloaded and verified


ODENVP(
  (transforms): ModuleList(
    (0): StackedCNFLayers(
      (chain): ModuleList(
        (0): LogitTransform()
        (1): CNF_Gate(
          (gate_net): FeedforwardGateI(
            (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
            (conv1): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu1): ReLU(inplace)
            (conv2): Conv2d(10, 10, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
            (bn2): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu2): ReLU(inplace)
            (avg_layer): AdaptiveAvgPool2d(output_size=(1, 1))
            (linear_layer): Conv2d(10, 1, kernel_size=(1, 1), stride=(1, 1))
            (sigmoid): Sigmoid()
          )
          (odefunc): RegularizedODEfunc(
            (odefunc): ODEfu

15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0


/tancode/repos/tan-ffjord/lib/layers/odefunc_rl.py:291: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  t = torch.tensor(t).type_as(y)
/opt/conda/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Iter 0010 | Time 11.4602(29.3219) | Bit/dim 8.9022(9.0696) | Xent 2.2766(2.2999) | Loss 20.6032(21.1800) | Error 0.7511(0.8716) Steps 466(460.91) | Grad Norm 20.4550(29.4829) | Total Time 0.00(0.00)
Iter 0020 | Time 11.2595(24.6480) | Bit/dim 8.5315(8.9471) | Xent 2.2347(2.2872) | Loss 19.9874(20.8880) | Error 0.7667(0.8420) Steps 442(455.78) | Grad Norm 9.1356(24.9165) | Total Time 0.00(0.00)
Iter 0030 | Time 11.1676(21.1194) | Bit/dim 8

...

Iter 6770 | Time 20.0060(19.6075) | Bit/dim 3.6169(3.6273) | Xent 0.5305(0.5050) | Loss 9.1473(9.9017) | Error 0.1922(0.1796) Steps 730(753.26) | Grad Norm 12.1683(8.9461) | Total Time 0.00(0.00)
Iter 6780 | Time 20.4246(19.6194) | Bit/dim 3.6267(3.6296) | Xent 0.5934(0.5091) | Loss 9.2257(9.6958) | Error 0.2044(0.1799) Steps 748(752.01) | Grad Norm 17.4034(9.5726) | Total Time 0.00(0.00)
Iter 6790 | Time 19.7315(19.6489) | Bit/dim 3.6260(3.6313) | Xent 0.4681(0.5054) | Loss 9.1265(9.5366) | Error 0.1711(0.1793) Steps 772(753.27) | Grad Norm 7.4764(9.2392) | Total Time 0.00(0.00)
Iter 6800 | Time 20.3562(19.6704) | Bit/dim 3.5973(3.6319) | Xent 0.5024(0.5020) | Loss 8.9815(9.4063) | Error 0.1711(0.1777) Steps 778(753.30) | Grad Norm 4.5518(8.5323) | Total Time 0.00(0.00)
Iter 6810 | Time 19.8659(19.6467) | Bit/dim 3.6649(3.6310) | Xent 0.4918(0.4943) | Loss 9.2012(9.3143) | Error 0.1733(0.1755) Steps 730(750.04) | Grad Norm 11.9036(8.1582) | Total Time 0.00(0.00)
Iter 6820 | Time 20.85

11.2874251497006
11.2874251497006
11.2874251497006
11.2874251497006
11.2874251497006
11.2874251497006
11.2874251497006
11.2874251497006
11.2874251497006
11.2874251497006
11.2874251497006
11.2874251497006
11.2874251497006
11.2874251497006


Iter 6830 | Time 18.7369(19.7714) | Bit/dim 3.6368(3.6289) | Xent 0.4791(0.4962) | Loss 9.0217(9.7857) | Error 0.1567(0.1754) Steps 742(754.21) | Grad Norm 10.0675(8.1144) | Total Time 0.00(0.00)
Iter 6840 | Time 19.3938(19.8969) | Bit/dim 3.6476(3.6261) | Xent 0.4822(0.4869) | Loss 8.9481(9.5731) | Error 0.1678(0.1719) Steps 748(757.78) | Grad Norm 8.2791(7.7737) | Total Time 0.00(0.00)
Iter 6850 | Time 20.1908(19.9039) | Bit/dim 3.6530(3.6259) | Xent 0.4216(0.4846) | Loss 8.9951(9.4450) | Error 0.1456(0.1715) Steps 730(758.02) | Grad Norm 6.4516(7.5714) | Total Time 0.00(0.00)
Iter 6860 | Time 19.8502(19.8453) | Bit/dim 3.6023(3.6269) | Xent 0.5028(0.4837) | Loss 8.9827(9.3430) | Error 0.1811(0.1724) Steps 766(760.00) | Grad Norm 5.5543(7.6030) | Total Time 0.00(0.00)
Iter 6870 | Time 19.6140(19.7442) | Bit/dim 3.6158(3.6268) | Xent 0.5393(0.4895) | Loss 9.0772(9.2712) | Error 0.1900(0.1741) Steps 760(758.38) | Grad Norm 9.0492(7.6531) | Total Time 0.00(0.00)
validating...
Epoch 0125

11.25748502994012
11.25748502994012
11.25748502994012
11.25748502994012
11.25748502994012
11.25748502994012
11.25748502994012
11.25748502994012
11.25748502994012
11.25748502994012
11.25748502994012
11.25748502994012
11.25748502994012
11.25748502994012


Iter 6880 | Time 19.2789(19.7681) | Bit/dim 3.6122(3.6257) | Xent 0.4579(0.4896) | Loss 8.9948(9.9264) | Error 0.1633(0.1737) Steps 766(759.78) | Grad Norm 7.6395(7.7792) | Total Time 0.00(0.00)
Iter 6890 | Time 20.8399(19.8542) | Bit/dim 3.6147(3.6237) | Xent 0.5242(0.4839) | Loss 9.0381(9.6869) | Error 0.1856(0.1721) Steps 778(762.59) | Grad Norm 10.5983(7.9977) | Total Time 0.00(0.00)
Iter 6900 | Time 19.4741(19.9020) | Bit/dim 3.6386(3.6231) | Xent 0.4555(0.4849) | Loss 8.9962(9.5166) | Error 0.1567(0.1721) Steps 730(762.80) | Grad Norm 7.1127(8.1390) | Total Time 0.00(0.00)
Iter 6910 | Time 20.1182(19.8185) | Bit/dim 3.6202(3.6247) | Xent 0.4370(0.4846) | Loss 9.0698(9.3985) | Error 0.1544(0.1721) Steps 760(761.58) | Grad Norm 7.4670(8.1138) | Total Time 0.00(0.00)
Iter 6920 | Time 20.3007(19.8958) | Bit/dim 3.6074(3.6276) | Xent 0.4473(0.4803) | Loss 9.0390(9.3162) | Error 0.1667(0.1707) Steps 748(763.38) | Grad Norm 6.1129(7.7636) | Total Time 0.00(0.00)
